$y_i = \beta\cdot x_i + \alpha + \varepsilon_i$

In [1]:
def predict(alpha, beta, x_i):
    return beta * x_i + alpha

In [2]:
def error(alpha, beta, x_i, y_i):
    """
    the error from predicting beta * x_i + alpha
    when the actual value is y_i
    """
    return y_i - predict(alpha, beta, x_i)

In [3]:
def sum_of_squared_errors(alpha, beta, x, y):
    return sum(error(alpha, beta, x_i, y_i) ** 2 
               for x_i, y_i in zip(x, y))

"The *least squares* solution is to choose the alpha and beta that make sum_of_squared_errors as small as possible."

In [6]:
import math

def dot(v, w):
    """
    v_1 * w_1 + ... + v_n * w_n
    """
    return sum(v_i * w_i 
               for v_i, w_i in zip(v, w))

def mean(x):
    return sum(x) / len(x)

def sum_of_squares(v):
    """v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

def variance(x):
    """
    assumes x has at least two elements
    """
    n = len(x)
    deviations = de_mean(x)
    return sum_of_squares(deviations) / (n - 1)

def standard_deviation(x):
    return math.sqrt(variance(x))

def de_mean(x):
    """
    translate x by subtracting its mean (so the result has mean 0)
    """
    x_bar = mean(x)
    return [x_i - x_bar for x_i in x]

def covariance(x, y):
    n = len(x)
    return dot(de_mean(x), de_mean(y)) / (n - 1)

def correlation(x, y):
    stdev_x = standard_deviation(x)
    stdev_y = standard_deviation(y)
    if stdev_x > 0 and stdev_y > 0:
        return covariance(x, y) / stdev_x / stdev_y
    else:
        return 0 # if no variation, correlation is zero

def least_squares_fit(x, y):
    """
    given training values for x and y,
    find the least-squares values of alpha and beta
    """
    beta = correlation(x, y) * standard_deviation(y) / standard_deviation(x)
    alpha = mean(y) - beta * mean(x)
    return alpha, beta

* "The choice of alpha simply says that when we see the average value of the independent variable x , we predict the average value of the dependent variable y".
* "The choice of beta means that when the input value increases by standard_deviation(x) , the prediction increases by correlation(x, y) * standard_deviation(y)".
    - "In the case when x and y are perfectly correlated, a one standard deviation increase in x results in a one-standard-deviation-of- y increase in the prediction."
    - "When they’re perfectly anticorrelated, the increase in x results in a decrease in the prediction."
    - "And when the correlation is zero, beta is zero, which means that changes in x don’t affect the prediction at all."

"Of course, we need a better way to figure out how well we’ve fit the data than staring at the graph. A common measure is the coefficient of determination (or R-squared), which measures the fraction of the total variation in the dependent variable that is
captured by the model:"

In [1]:
def total_sum_of_squares(y):
    """
    the total squared variation of y_i's from their mean
    """
    return sum(v ** 2 for v in de_mean(y))

def r_squared(alpha, beta, x, y):
    """
    the fraction of variation in y captured by the model, 
    which equals 1 - the fraction of variation in y not captured 
    by the model
    """
    return 1.0 - (sum_of_squared_errors(alpha, beta, x, y) / 
                  total_sum_of_squares(y))

"Now, we chose the alpha and beta that minimized the sum of the squared prediction errors."

"One linear model we could have chosen is “always predict mean(y) ” (corresponding to alpha = mean(y) and beta = 0 ), whose sum of squared errors exactly equals its total sum of squares.
* This means an R-squared of zero, which indicates a model that (obviously, in this case) performs no better than just predicting the mean."
* "Clearly, the least squares model must be at least as good as that one, which means that the sum of the squared errors is at most the total sum of squares, which means that the R-squared must be at least zero. And the sum of squared errors must be at least 0, which means that the R-squared can be at most 1."

#### Using Gradient Descent

"If we write theta = \[alpha, beta\] , then we can also solve this using gradient descent:"

In [2]:
def squared_error(x_i, y_i, theta):
    alpha, beta = theta
    return error(alpha, beta, x_i, y_i) ** 2

def squared_error_gradient(x_i, y_i, theta):
    alpha, beta = theta
    return [-2 * error(alpha, beta, x_i, y_i), # alpha partial derivative
            -2 * error(alpha, beta, x_i, y_i) * x_i] # beta partial derivative